# **Phoneme Classification**

## **Download Data**
Download data from google drive,then unzip it


In [1]:
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls

Downloading...
From: https://drive.google.com/uc?id=1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR
To: /content/data.zip
372MB [00:04, 87.0MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
data.zip  sample_data  timit_11


## **Preparing Data**

In [2]:
import numpy as np

print('Loading data ...')

data_root = './timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data:{}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data:(451552, 429)


## **Create Dataset**

In [3]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
  def __init__(self, X, y=None):
    self.data = torch.from_numpy(X).float()
    if y is not None:
      y = y.astype(np.int)
      self.label = torch.LongTensor(y)
    else:
      self.label = None
    
  def __getitem__(self, idx):
    if self.label is not None:
      return self.data[idx], self.label[idx]
    else:
      return self.data[idx]
  
  def __len__(self):
    return len(self.data)

In [4]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]

print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


In [5]:
BATCH_SIZE = 64
from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

In [6]:
import gc
del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

283

## **Create Model**

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
  def __init__(self):
    super(Classifier, self).__init__()
    self.layer1 = nn.Linear(429, 1024)
    self.layer2 = nn.Linear(1024, 512)
    self.layer3 = nn.Linear(512, 128)
    self.out = nn.Linear(128, 39)

    self.act_fn = nn.Sigmoid()

  def forward(self, x):
    x = self.layer1(x)
    x = self.act_fn(x)

    x = self.layer2(x)
    x = self.act_fn(x)

    x = self.layer3(x)
    x = self.act_fn(x)

    x = self.out(x)

    return x 

In [13]:
# 模型的第二种写法
import torch
import torch.nn as nn

class Classifier1(nn.Module):
  def __init__(self):
    super(Classifier1, self).__init__()

    self.net = nn.Sequential(
        nn.Linear(429, 1024),
        nn.Sigmoid(),
        nn.Linear(1024, 512),
        nn.Sigmoid(),
        nn.Linear(512, 128),
        nn.Sigmoid(),
        nn.Linear(128, 39),
        )
    self.criterion = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.net(x).squeeze(1)
  
  def cal_loss(self, pred, target):
    return self.criterion(pred, target)

## **Training**

In [9]:
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
# fix random seed
def same_seeds(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  np.random.seed(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True

In [ ]:
same_seeds(0)

device = get_device()
print(f'DEVICE: {device}')

num_epoch = 20
learning_rate = 0.0001

model_path = './model.cpkt'

model = Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/020] Train Acc: 0.794340 Loss: 0.623927 | Val Acc: 0.700439 loss: 0.966146
saving model with acc 0.700
[002/020] Train Acc: 0.797811 Loss: 0.611550 | Val Acc: 0.696667 loss: 0.977135
[003/020] Train Acc: 0.801968 Loss: 0.600017 | Val Acc: 0.698517 loss: 0.977965
[004/020] Train Acc: 0.805116 Loss: 0.588905 | Val Acc: 0.697675 loss: 0.984102
[005/020] Train Acc: 0.808756 Loss: 0.577875 | Val Acc: 0.695240 loss: 0.994218
[006/020] Train Acc: 0.811913 Loss: 0.567292 | Val Acc: 0.698334 loss: 1.003270
[007/020] Train Acc: 0.815399 Loss: 0.556826 | Val Acc: 0.694907 loss: 1.008838
[008/020] Train Acc: 0.818834 Loss: 0.546317 | Val Acc: 0.694281 loss: 1.030975
[009/020] Train Acc: 0.821696 Loss: 0.536533 | Val Acc: 0.694000 loss: 1.022089
[010/020] Train Acc: 0.825182 Loss: 0.526311 | Val Acc: 0.695110 loss: 1.032463
[011/020] Train Acc: 0.827850 Loss: 0.516963 | Val Acc: 0.694529 loss: 1.031812
[012/020] Train Acc: 0.830807 Loss: 0.507770 | Val Acc: 0.692423 loss: 1.050617
[013/020] Tr

**-----------分界-----------**

利用classifier1训练模型

In [14]:
same_seeds(0)

device = get_device()
print(f'DEVICE: {device}')

num_epoch = 20
learning_rate = 0.0001

model_path = './model.cpkt'

model = Classifier1().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = model.cal_loss(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = model.cal_loss(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/020] Train Acc: 0.467302 Loss: 1.811661 | Val Acc: 0.567428 loss: 1.433136
saving model with acc 0.567
[002/020] Train Acc: 0.594390 Loss: 1.330549 | Val Acc: 0.628952 loss: 1.215165
saving model with acc 0.629
[003/020] Train Acc: 0.644405 Loss: 1.154058 | Val Acc: 0.659372 loss: 1.100344
saving model with acc 0.659
[004/020] Train Acc: 0.672454 Loss: 1.051853 | Val Acc: 0.675837 loss: 1.039374
saving model with acc 0.676
[005/020] Train Acc: 0.691067 Loss: 0.983493 | Val Acc: 0.685032 loss: 1.001758
saving model with acc 0.685
[006/020] Train Acc: 0.705082 Loss: 0.932414 | Val Acc: 0.689723 loss: 0.979979
saving model with acc 0.690
[007/020] Train Acc: 0.716166 Loss: 0.891978 | Val Acc: 0.692317 loss: 0.967356
saving model with acc 0.692
[008/020] Train Acc: 0.725582 Loss: 0.858017 | Val Acc: 0.695614 loss: 0.958754
saving model with acc 0.696
[009/020] Train Acc: 0.733844 Loss: 0.829526 | Val Acc: 0.694163 loss: 0.963273
[010/020] Train Acc: 0.741346 Loss: 0.803827 | Val Acc: 

## **Testing**

In [ ]:
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [ ]:
predict = []
model.eval()

with torch.no_grad():
  for i, data in enumerate(test_loader):
    inputs = data
    inputs = inputs.to(device)
    outputs = model(inputs)
    _, test_pred = torch.max(outputs, dim=1)

    for y in test_pred.cpu().numpy():
      predict.append(y)

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))